In [1]:
import pandas as pd

In [4]:
#leitura dos dados
df_review = pd.read_csv("IMDB Dataset.csv")
#checagem do balanceamento
df_review.value_counts(subset='sentiment')

sentiment
negative    25000
positive    25000
Name: count, dtype: int64

Dataset já veio perfeitamente balanceado. Nesse caso, não há necessidade de tomar qualquer ação nesse sentido.